In [1]:
#####################################   Importing data   #########################################################################################

import numpy as np
from vpython import *
%matplotlib notebook
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display
from matplotlib.widgets import Slider, Button
from tkinter import *

###############################################################################################################################

<IPython.core.display.Javascript object>

In [2]:
########neutrino function########

# mixing factors (converted from degrees to radians)
th12 = np.radians(33.82)
th23 = np.radians(48.3)
th13 = np.radians(8.61)
delta_CP = np.radians(222)

# mass differences
dm21_sq = 7.53*10**(-5) # eV^2
dm32_sq = 2.52*10**(-3) # eV^2

# mass eigenstates
m1 = 0.75 # eV
m2 = np.sqrt(m1*m1 + dm21_sq) # eV
m3 = np.sqrt(m2*m2 + dm32_sq) # eV

#calculate third mass difference squared
dm31_sq = m3*m3 - m1*m1 # eV^2

# set energy values
E_sun_pp = 4*10**5 # eV 
E_atm=10**11 #eV
E_acc=10**9 #eV

# important constants:
# speed of light
c = 299792.458 # km s^-1 
# Planck constant
hbar = 6.582119569*10**(-16) # eV*s
# Astronomical unit (i.e. the distance between the Sun and the Earth)
AU = 149597870.700 # km



def prob_transition(alpha, beta, E, L):
    """Calculates the probability of a transition between an initial neutrino alpha to a neutrino beta
    Takes as inputs the initial and final neutrino flavours alpha and beta, 
    the energy at which the neutrino is produced E (in eV),
    and the distance travelled by the particle L (in km)
    Returns the probability of the transition"""
    
    if (alpha == "electron" and beta == "muon") or (alpha == "muon" and beta == "electron"): 
        ar = np.array([th12, dm21_sq])
        arg = 1.27*ar[1]*L/E
        P = np.sin(2*ar[0])*np.sin(2*ar[0])*np.sin(arg)*np.sin(arg)
        
    elif (alpha == "electron" and beta == "tau") or (alpha == "tau" and beta == "electron"):
        ar = np.array([th13, dm31_sq])
        arg = 1.27*ar[1]*L/E
        P = np.sin(2*ar[0])*np.sin(2*ar[0])*np.sin(arg)*np.sin(arg)
        
    elif (alpha == "muon" and beta == "tau") or (alpha == "tau" and beta == "muon"):
        ar = np.array([th23, dm32_sq])
        arg = 1.27*ar[1]*L/E
        P = np.sin(2*ar[0])*np.sin(2*ar[0])*np.sin(arg)*np.sin(arg)
        
    elif (alpha == beta):
        
        if (alpha == "electron"):
            ar1 = np.array([th12, dm21_sq])
            ar2 = np.array([th13, dm31_sq])
            
        elif (alpha == "muon"):
            ar1 = np.array([th12, dm21_sq])
            ar2 = np.array([th23, dm32_sq])
            
        elif (alpha == "tau"):
            ar1 = np.array([th13, dm31_sq])
            ar2 = np.array([th23, dm32_sq])
        
        arg1 = 1.27*ar1[1]*L/E
        arg2 = 1.27*ar2[1]*L/E
        
        P1 = np.sin(2*ar1[0])*np.sin(2*ar1[0])*np.sin(arg1)*np.sin(arg1)
        P2 = np.sin(2*ar2[0])*np.sin(2*ar2[0])*np.sin(arg2)*np.sin(arg2)
        
        if ((P1 + P2) < 1) :
            P = 1 - (P1 + P2)
            
        else:
            P = 0
    
    else:
        return "Error"
    
    return P

In [3]:
########################################   GAME   ############################################################################################################################################


####### Setting up initial data #######

g = 30               # gravitational acceleration (m.s^-2)
mass_bird = 1        # mass of the bird (kg)
radius_bird = 0.2    # radius of the bird (m)
x0 = 0.0             # initial x-coordinate of the bird
y0 = 0.6 + radius_bird # initial y-coordinate of the bird
p = 200              # density of the target (kg.m^-3) ~ density of a balsa type of wood
contact_time = 0.01  # contact time of the impact between the bird and the target (sec)
dt = 0.01            # time interval for the animations (sec)
d_angle = 0.01       # angle interval for the animation of the toppling of the target (rad)
t_applied = 0        # setting up the initial value of the applied torque for the while loop
v0=10
theta=45

bird_color = [color.blue,color.green,color.red]
bird_mass_topple = [0.3,1,1.8]



##### Defining the function of the trajectory y(x) #####

def trajectory(x_pos):
    '''
    Calculates the trajectory (y coordinate) at a given horizontal distance (x coordinate) from the lauch point.
    
    Input: 
     - x_pos: x coordinate of the point where the trajectory is required.
     
    Output:
     - trajectory: y coordinate of the particle at this point.
    '''
    
    trajectory = -g*(x_pos-x0)**2 / (2*v0**2*np.cos(theta)**2) + np.tan(theta)*(x_pos-x0) + y0
    
    return trajectory    


####### Main code of the game ####### 

V_center = vector(8,0,0)
V_camera = vector(4,2,12)
V_title = vector(2,6,-2)

# Setting up the scene 
scene = canvas(width=1000, height=700, center=V_center, range=8, background = color.cyan) # scene where the game animations will be displayed
scene.camera.pos = V_camera                                                               # position of the camera (from where the player sees the game)
scene.forward = V_center - scene.camera.pos                                               # axis of the camera (in which direction the player is looking)
scene.camera.axis = V_center - scene.camera.pos                                           # axis of the camera (in which direction the player is looking)
scene.userzoom = False
scene.userspin = False
    
# positioning the ground of the animation
ground = box(pos= V_center, length=100, height=0.1, width=100, texture = textures.metal)
    
# positioning the target at a random position
target = box(pos=vector((2*random()+1)*7,1,0), axis=vector(1,0,0), length= 0.5, height= 3 + random(), width=0.5, color = color.black)

# box
pedestal = box(pos = vector(0,0.35,0), length = 1, width = 2, height = 0.5, texture = textures.wood)

# positioning the bird on the origin of the scene
global bird
bird = sphere(pos = vector(x0,y0,0), radius = 0.2, color=color.blue, make_trail = True, trail_color = color.blue)

# Text
label_title = label(text = 'ANGRY NEUTRINOS',pos = vector(8,7,0), color = color.black, box = False, background = color.cyan , height = 40, opacity = 0)
label_level = label(text = 'Level 1' , pos = vector(8,6,0), color = color.black, box = False, background = color.cyan , height = 30, opacity = 0)
      
# Initial data with respect to the scene set up 
mass_target = p*(target.length*target.width*target.height)  # setting up the mass of the target (m = p*V)
hit_tolerance = radius_bird + target.length/2               # computing the value of the hit tolerance : x-distance between bird.pos and target.pos at the time of the collision
x_impact = target.pos.x - hit_tolerance                     # x-coordinate of the impact point
t_restoring = mass_target*g*target.length/2                 # computing the magnitude of the restoring torque of the target

global c
c = points(radius = 3)   
 
def vector_list():   
    global v0
    global theta
    Vectors=[]
    xpoints_slider=np.linspace(0,3,15)
    for x in xpoints_slider:
        Vectors.append(vector(x,trajectory(x),0))
    return Vectors




#sliders to adjust speed and angle
def setV(v):
    global v0
    wt1.text = '{:1.0f}'.format(v.value)
    v0 = v.value
    c.clear()
    c.append(vector_list())
    
sl1 = slider(min = 10 , max = 30 , value = 10, length = 220 , bind = setV, right = 15)
wt1 = wtext(text = '{:1.0f}'.format(sl1.value)) 
    
def setTheta(t):
    global theta
    wt2.text = '{:1.0f}'.format(t.value)
    theta = np.radians(t.value)
    c.clear()
    c.append(vector_list())
    
sl2 = slider(min = 0 , max = 90 , value = 45, length = 220 , bind = setTheta, right = 15)
wt2 = wtext(text = '{:1.0f}'.format(sl2.value)) 

def setT(t):
    runanim()

sl3 = slider(min = 0 , max = 1, value = 0, length = 100 , bind = setT)
wt3 = wtext(text = "GO")
    
def runanim():
    
    global bird
    global v0
    global theta
    global target
    global sl3
    global c
    
    c.clear()
    
    x = x0
    y = y0
    t = 0
    distance_covered = 0
            
    # while the ball hasn't reached the target
    while x <= x_impact and y > 0 :
                
        # time increases by dt
        rate(25)
        t += dt

        distance_covered += np.sqrt(abs(x - (x0 + v0*t*np.cos(theta)))**2 + abs(y - (y0 + v0*t*np.sin(theta) - 0.5*g*t**2))**2)

        prob_elec = prob_transition("muon", "electron", E_sun_pp , distance_covered*1.5*AU)
        prob_muon = prob_transition("muon", "muon", E_sun_pp , distance_covered*1.5*AU)
        prob_tau = prob_transition("muon", "tau", E_sun_pp , distance_covered*1.5*AU)

        prob = [prob_muon,prob_tau,prob_elec]
        prob_max = np.amax(prob)
        index_max = prob.index(prob_max)

        mass_topple = bird_mass_topple[index_max]
        bird.color = bird_color[index_max]
        bird.trail_color = bird_color[index_max]

        x = x0 + v0*t*np.cos(theta)  # new x-position of the bird
        y = y0 + v0*t*np.sin(theta) - 0.5*g*t**2  # new y-position of the bird
        bird.pos = vector(x,y,0)     # updating the position of the bird
    
    # if the bird collides with the target
    if target.height + bird.radius > y > 0 :

        c_rot = target.pos + vector(target.length/2,-target.height/2,0)   # center of rotation position
        bird_momentum = vector(mass_bird*v0*np.cos(theta) , mass_bird*v0*np.sin(theta) - mass_bird*g*t , 0) # momentum of the bird at the time of the impact
        vector_da = vector(x_impact,trajectory(x_impact),0) - c_rot       # vector from the point of rotation to the point of impact
        t_applied_vector = cross(bird_momentum/contact_time,vector_da)    # torque applied by the bird on the target
        t_applied = mag(t_applied_vector)                                 # magnitude of the torque applied 
            
        # if the applied torque is enough to topple the target 
        if t_applied > t_restoring :

            s = target.pos
            w = bird.pos
            alpha = 0      # anticlockwise angle of the horizontal axis to the axis of the target                         
            betha = np.pi - np.arccos((target.length/2)/mag(s-c_rot))                    # anticlockwise angle of the horizontal axis to the vector from the center of rotation to the position of the target 
            gamma = np.pi - np.arccos((hit_tolerance + target.length/2) / mag(w-c_rot))  # anticlockwise angle of the horizontal axis to the vector from the center of rotation to the position of the bird

            # while the toppling of the target isn't finished
            while alpha >= -np.pi/2:
                rate(25)
                betha -= d_angle    # the angle decreases by d_angle
                alpha -= d_angle    # '        '
                gamma -= d_angle    # '        '
                o = vector(np.cos(alpha),np.sin(alpha),0)                       # computing the new axis vector of the target
                u = c_rot + vector(np.cos(betha),np.sin(betha),0)*mag(s-c_rot)  # computing the new position vector of the target
                z = c_rot + vector(np.cos(gamma),np.sin(gamma),0)*mag(w-c_rot)  # computing the new position vector of the bird
                target.pos = u        # updating the position of the target 
                target.axis = o      # updating the axis of the target
                bird.pos = z         # updating the position of the bird
                    
        # if the applied torque isn't enough to topple the target --> falling of the bird following a vertical straight line
        else :
                
            # while the bird is still above the ground 
            while y > 0 :
                    
                rate(25)
                t += dt                                     # time increases by dt                    
                y = -0.5*g*t**2 + trajectory(x)             # new y-position of the bird
                bird.pos = vector(x,y,0)                    # updating the position of the bird

    # if the bird doesn't collide with the target 
    else :

        # while the bird is above the ground 
        while x < 35 and y >= 0:
                
            # time increases by dt
            rate(25)
            t += dt

            distance_covered += np.sqrt(abs(x - (x0 + v0*t*np.cos(theta)))**2 + abs(y - (y0 + v0*t*np.sin(theta) - 0.5*g*t**2))**2)

            prob_elec = prob_transition("muon", "electron", E_sun_pp , distance_covered*1.5*AU)
            prob_muon = prob_transition("muon", "muon", E_sun_pp , distance_covered*1.5*AU)
            prob_tau = prob_transition("muon", "tau", E_sun_pp , distance_covered*1.5*AU)

            prob = [prob_muon,prob_tau,prob_elec]
            prob_max = np.amax(prob)
            index_max = prob.index(prob_max)

            mass_topple = bird_mass_topple[index_max]
            bird.color = bird_color[index_max]
            bird.trail_color = bird_color[index_max]

            x = x0 + v0*t*np.cos(theta)  # new x-position of the bird
            y = y0 + v0*t*np.sin(theta) - 0.5*g*t**2  # new y-position of the bird
            bird.pos = vector(x,y,0)     # updating the position of the bird
      
    # Initializing
    bird.clear_trail()
    bird.make_trail = False
    bird.color = color.blue
    bird.pos = vector(x0,y0,0)
    bird.make_trail = True
    
    sl3.value = 0
    

##################################################################################################################################################################################


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>